# Neon Demo: Near-Equality and Tolerance Arithmetic

Interactive demonstration of elemental-neon's floating-point comparison and tolerance features.

In [ ]:
# Install neon
!pip install elemental-neon matplotlib numpy

In [ ]:
from neon import compare, clamp, safe, ulp, inspect
import math

## Problem 1: The 0.1 + 0.2 != 0.3 Bug

Everyone knows this one...

In [ ]:
# Naive comparison FAILS
print(f"0.1 + 0.2 == 0.3: {0.1 + 0.2 == 0.3}")  # False!
print(f"Actual value: {0.1 + 0.2}")
print(f"Difference: {(0.1 + 0.2) - 0.3}")

# Neon handles it correctly
print(f"\ncompare.near(0.1 + 0.2, 0.3): {compare.near(0.1 + 0.2, 0.3)}")  # True!

## Problem 2: Near-Zero Comparisons

Should 1e-16 be considered "close enough" to zero?

In [ ]:
tiny_value = 1e-16

# Naive comparison
print(f"Naive: {tiny_value} == 0.0? {tiny_value == 0.0}")

# With neon
print(f"Neon: compare.near_zero({tiny_value})? {compare.near_zero(tiny_value)}")
print(f"Neon: compare.near({tiny_value}, 0.0)? {compare.near(tiny_value, 0.0)}")

## Feature: Relative vs Absolute Tolerance

Different use cases need different comparison strategies.

In [ ]:
# Relative tolerance: Good for large numbers (percentage-based)
print("Relative tolerance (1% = 0.01):")
print(f"  1000.0 vs 1001.0: {compare.near_rel(1000.0, 1001.0, tol=0.01)}")
print(f"  1000.0 vs 1010.0: {compare.near_rel(1000.0, 1010.0, tol=0.01)}")
print(f"  1000.0 vs 1011.0: {compare.near_rel(1000.0, 1011.0, tol=0.01)}")

# Absolute tolerance: Good for near-zero (fixed threshold)
print("\nAbsolute tolerance (fixed at 0.01):")
print(f"  0.001 vs 0.005: {compare.near_abs(0.001, 0.005, tol=0.01)}")
print(f"  0.001 vs 0.012: {compare.near_abs(0.001, 0.012, tol=0.01)}")

## Feature: Safe Division

Handle division by zero gracefully.

In [ ]:
# Division with custom default
print("safe.div(10, 0, default=0.0):", safe.div(10, 0, default=0.0))
print("safe.div(10, 0, default=None):", safe.div(10, 0, default=None))

# Return zero on division by zero
print("\nsafe.div_or_zero(10, 0):", safe.div_or_zero(10, 0))

# Return infinity on division by zero
print("\nsafe.div_or_inf(10, 0):", safe.div_or_inf(10, 0))
print("safe.div_or_inf(-10, 0):", safe.div_or_inf(-10, 0))
print("safe.div_or_inf(0, 0):", safe.div_or_inf(0, 0))  # NaN

## Feature: Snapping Values

Clean up near-zero values or snap to specific targets.

In [ ]:
# Snap near-zero values to exactly zero
messy_data = [1e-16, 0.5, -1e-15, 1.0, 3e-17]
cleaned = clamp.to_zero_many(messy_data)
print(f"Original: {messy_data}")
print(f"Cleaned:  {cleaned}")

# Snap to nearest integer if close
print(f"\nclamp.to_int(2.9999999999): {clamp.to_int(2.9999999999)}")
print(f"clamp.to_int(2.5): {clamp.to_int(2.5)}")

# Snap to specific value
print(f"\nclamp.to_value(0.333333333, 1/3): {clamp.to_value(0.333333333, 1/3)}")

## Feature: Exact Summation

Use `math.fsum()` for maximum precision.

In [ ]:
# Adding 0.1 ten times
values = [0.1] * 10

naive_sum = sum(values)
exact_sum = safe.sum_exact(values)

print(f"Naive sum: {naive_sum}")
print(f"Exact sum: {exact_sum}")
print(f"Difference: {naive_sum - 1.0}")
print(f"Exact is 1.0? {exact_sum == 1.0}")

## Feature: ULP (Unit in the Last Place) Operations

Work at the precision of floating-point representation.

In [ ]:
# What's the ULP of 1.0?
print(f"ULP of 1.0: {ulp.of(1.0)}")
print(f"ULP of 1e10: {ulp.of(1e10)}")

# Next/previous representable float
x = 1.0
next_float = ulp.next(x)
prev_float = ulp.prev(x)

print(f"\nValue: {x}")
print(f"Next:  {next_float}")
print(f"Prev:  {prev_float}")

# How far apart are they in ULPs?
print(f"\nULP distance: {ulp.diff(x, next_float)}")

# Check if within N ULPs
print(f"\nWithin 4 ULPs? {ulp.within(1.0, ulp.add(1.0, 4))}")
print(f"Within 4 ULPs? {ulp.within(1.0, ulp.add(1.0, 5))}")

## Visualization: Floating-Point Precision Limits

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Show how ULP size changes with magnitude
magnitudes = np.logspace(-10, 10, 100)
ulp_sizes = [ulp.of(float(m)) for m in magnitudes]

plt.figure(figsize=(12, 6))
plt.loglog(magnitudes, ulp_sizes, linewidth=2)
plt.xlabel('Magnitude', fontsize=12)
plt.ylabel('ULP Size', fontsize=12)
plt.title('Floating-Point Precision vs Magnitude', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"At 1.0:    ULP = {ulp.of(1.0):.2e}")
print(f"At 1e6:    ULP = {ulp.of(1e6):.2e}")
print(f"At 1e-6:   ULP = {ulp.of(1e-6):.2e}")

## Real-World Example: Financial Calculations

Handling penny rounding errors.

In [ ]:
# Calculating total from many small transactions
transactions = [0.01] * 300  # 300 pennies

naive_total = sum(transactions)
exact_total = safe.sum_exact(transactions)

print(f"Expected: $3.00")
print(f"Naive sum: ${naive_total:.10f}")
print(f"Exact sum: ${exact_total:.10f}")
print(f"\nError with naive: {(naive_total - 3.0) * 100:.6f} cents")
print(f"Error with exact: {(exact_total - 3.0) * 100:.6f} cents")

## Real-World Example: Percentage Comparisons

In [ ]:
# Check if stock prices are within 1% of target
target_price = 150.00
tolerance = 0.01  # 1%

prices = [149.00, 150.50, 151.00, 151.60]

print(f"Target: ${target_price:.2f} ± {tolerance*100}%")
print("\nPrices:")
for price in prices:
    within_range = compare.near_rel(target_price, price, tol=tolerance)
    diff_pct = abs(price - target_price) / target_price * 100
    status = "✓" if within_range else "✗"
    print(f"  ${price:.2f} ({diff_pct:.2f}% diff) {status}")

## Summary

Neon provides:
- ✅ Correct floating-point comparison with tolerance
- ✅ **NEW v1.1.0**: Production debugging tools (`inspect` module)
- ✅ **NEW v1.1.0**: FP8/FP16 quantization validation for AI/ML
- ✅ Safe arithmetic (division, sqrt, log, etc.)
- ✅ **NEW v1.1.0**: Batch operations (`*_many()` functions)
- ✅ Value snapping and cleaning
- ✅ Exact summation with `math.fsum()`
- ✅ ULP-level precision control
- ✅ Zero dependencies (stdlib only)

Perfect for financial calculations, scientific computing, AI/ML quantization, and anywhere floating-point precision matters.

In [ ]:
# Batch safe division
numerators = [10, 20, 30, 40]
denominators = [2, 0, 5, 0]
results = safe.div_many(numerators, denominators, default=0.0)
print(f"Batch division: {results}")

# Batch near-zero check
values = [1e-15, 0.5, -1e-16, 1.0, 2e-17]
is_near_zero = compare.near_zero_many(values)
print(f"\nNear zero? {is_near_zero}")

# Batch snap to integer
floats = [2.9999999, 3.1, 5.0, 7.0000001]
snapped = clamp.to_int_many(floats)
print(f"\nSnapped to int: {snapped}")

# Batch ULP calculation
values = [1.0, 10.0, 100.0, 1000.0]
ulps = ulp.of_many(values)
print(f"\nULP sizes: {ulps}")

## ✨ NEW in v1.1.0: Batch Operations

Process collections efficiently with `*_many()` functions.

In [ ]:
# Simulate model weights
import numpy as np
np.random.seed(42)
weights = np.random.randn(1000).tolist()

# Add some problematic values
weights.extend([500.0, -600.0, 1e-10])  # Values that might overflow FP8

# Check if safe for FP8 (max ±448)
report = inspect.analyze_for_dtype(weights, target='fp8_e4m3')

print(f"Total values: {report.total}")
print(f"Safe: {report.safe}")
print(f"Overflow: {report.overflow}")
print(f"Underflow: {report.underflow}")
print(f"\n{report.recommendation}")

# Compare across multiple dtypes
comparison = inspect.compare_dtypes(weights, targets=['fp16', 'bf16', 'fp8_e4m3'])
print(f"\n{comparison.recommendation}")

### ✨ FP8/FP16 Quantization Validation (AI/ML Use Case)

Before quantizing models to low-precision formats, validate that values are safe.

In [ ]:
# Analyze a collection of values for issues
values = [1.0, 2.0, 5e-324, float('inf'), 3.0, float('nan')]
report = inspect.analyze(values)

print(report)
print(f"\nPrecision Risk: {report.precision_risk}")
print(f"\nIssues found:")
for issue in report.issues:
    print(f"  - {issue}")
print(f"\nRecommendations:")
for rec in report.recommendations:
    print(f"  - {rec}")

### Analyze Batch of Values

In [ ]:
# Debug the classic 0.1 + 0.2 != 0.3 issue
print(inspect.compare_debug(0.1 + 0.2, 0.3))

# Debug a larger difference
print("\n" + inspect.compare_debug(1.0, 2.0))

### Debug Why Floats Differ

In [ ]:
# Quick health check
result = 0.0 / 0.0  # NaN
if issue := inspect.check(result):
    print(f"Problem detected: {issue}")

# Check normal values (returns None)
print(f"\nNormal value check: {inspect.check(42.0)}")

# Check denormal (very tiny number)
denormal = 5e-324
if issue := inspect.check(denormal):
    print(f"\nDenormal warning: {issue}")

## ✨ NEW in v1.1.0: Production Debugging with `neon.inspect`

Debug floating-point issues in production with actionable recommendations.